In [1]:
import os
import typing
import logging
import pprint
import requests
from typing import Annotated, List
from dotenv import load_dotenv, find_dotenv
import tempfile
from pathlib import Path
import chromadb
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import autogen
from autogen.agentchat.contrib.web_surfer import WebSurferAgent
from autogen.coding.func_with_reqs import with_requirements
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent, UserProxyAgent
from autogen import register_function
from autogen.cache import Cache
from autogen.coding import LocalCommandLineCodeExecutor, CodeBlock

/Users/shibsankar/opt/anaconda3/envs/env_llm_agent/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set logging Configuration
logging.basicConfig(format='%(asctime)s %(levelname)s:%(name)s:%(message)s')
logger = logging.getLogger(__name__)
# Set a log level for the logger
logger.setLevel(logging.INFO)

# load environment variables from .env file
load_dotenv(find_dotenv())

# load parameters and configurations
config_list = [{
    "model": os.environ.get("OPENAI_DEPLOYMENT_NAME"),
    "api_key": os.environ.get("OPENAI_API_KEY"),
    "base_url": os.environ.get("OPENAI_ENDPOINT"),
    "api_version": os.environ.get("OPENAI_API_VERSION"),
    "api_type": "azure"
}]
llm_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.5
}
bing_api_key = os.environ.get("BING_API_KEY")

# Get the current project directory
current_dir = os.getcwd()
# Create a temporary file in the current directory with .txt extension
temp_file = tempfile.NamedTemporaryFile(dir=current_dir, suffix=".txt", delete=False)
# Store the file path in a variable
temp_file_path = temp_file.name
print(f"Temporary file path: {temp_file_path}")

Temporary file path: /Users/shibsankar/PycharmProjects/LLMAgent/notebook examples/tmp_6g24aj8.txt


In [3]:
# Define Tools
@with_requirements(python_packages=["typing", "requests", "autogen", "chromadb"], global_imports=["typing", "requests", "autogen", "chromadb"])
def rag_on_document(query: typing.Annotated[str, "The query to search in the index."], document: Annotated[Path, "Path to the document"]) -> str:
    logger.info(f"************  RAG on document is executed with query: {query} ************")
    default_doc = temp_file_path
    doc_path = default_doc if document is None or document == "" else document
    ragproxyagent = autogen.agentchat.contrib.retrieve_user_proxy_agent.RetrieveUserProxyAgent(
        "ragproxyagent",
        human_input_mode="NEVER",
        retrieve_config={
            "task": "qa",
            "docs_path": doc_path,
            "chunk_token_size": 1000,
            "model": config_list[0]["model"],
            "client": chromadb.PersistentClient(path="./tmp/chromadb"),
            "collection_name": "tourist_places",
            "get_or_create": True,
            "overwrite": False
        },
        code_execution_config={"use_docker": False}
    )
    res = ragproxyagent.initiate_chat(planner_agent, message=ragproxyagent.message_generator, problem = query, n_results = 2, silent=True)
    return str(res.chat_history[-1]['content'])


@with_requirements(python_packages=["typing", "requests", "autogen", "chromadb"], global_imports=["typing", "requests", "autogen", "chromadb"])
def bing_search(query: typing.Annotated[str, "The input query to search"]) -> Annotated[str, "The search results"]:
    web_surfer = WebSurferAgent(
        "bing_search",
        system_message="You are a Bing Web surfer Agent whose objective is to find relevant website urls for travel planning.",
        llm_config= llm_config,
        summarizer_llm_config=llm_config,
        browser_config={"viewport_size": 4096, "bing_api_key": bing_api_key}
    )
    register_function(
        visit_website,
        caller=web_surfer,
        executor=user_proxy,
        name="visit_website",
        description="This tool is to scrape content of website using a list of urls and store the website content into a text file that can be used for rag_on_document"
    )
    search_result = user_proxy.initiate_chat(web_surfer, message=query, summary_method="reflection_with_llm", max_turns=2)
    return str(search_result.summary)

def get_lat_lon(location):
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(location)

    if location:
        return location.latitude, location.longitude
    else:
        return None, None

@with_requirements(python_packages=["typing", "requests", "autogen", "chromadb"], global_imports=["typing", "requests", "autogen", "chromadb"])
def get_weather_info(destination: typing.Annotated[str, "The place of which weather information to retrieve"], start_date: typing.Annotated[str, "The date of the trip to retrieve weather data"]) -> typing.Annotated[str, "The weather data for given location"]:
    # Use Open-Meteo API to get weather forecast for the destination
    # Assuming the location of the destination is hardcoded for simplicity; in practice, use a geo-location API
    logger.info(f"************  Get weather API is executed for {destination}, {start_date} ************")
    coordinates = {"Grand Canyon": {"lat": 36.1069, "lon": -112.1129},
                   "Philadelphia": {"lat": 39.9526, "lon": -75.1652},
                   "Niagara Falls": {"lat": 43.0962, "lon": -79.0377},
                   "Goa": {"lat": 15.2993, "lon": 74.1240}}

    destination_coordinates = coordinates[destination]

    lat, lon = destination_coordinates["lat"], destination_coordinates["lon"] if destination in coordinates else get_lat_lon(destination)
    forecast_api_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&daily=temperature_2m_max,precipitation_sum&start={start_date}&timezone=auto"

    weather_response = requests.get(forecast_api_url)
    weather_data = weather_response.json()
    return str(weather_data)

@with_requirements(python_packages=["typing", "requests", "autogen", "chromadb"], global_imports=["typing", "requests", "autogen", "chromadb"])
def visit_website(urls: Annotated[List[str], "The list of url to scrape"], output_file: Annotated[str, "The path to the destination file to store scraped text"] = temp_file_path) -> Annotated[None, "This tools appends the scraped website into a text file"]:
    """
    Scrapes content from a list of URLs and saves the content into a .txt file.

    Parameters:
    - urls (list): List of URLs to scrape.
    - output_file (str): Name of the output file where scraped content is stored.
    """
    with open(output_file, 'w', encoding='utf-8') as file:
        for url in urls:
            try:
                print(f"Scraping: {url}")
                response = requests.get(url)
                response.raise_for_status()  # Ensure the request was successful

                # Parse the HTML content with BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')

                # Extract paragraphs from the HTML
                paragraphs = [p.get_text() for p in soup.find_all('p')]
                content = "\n".join(paragraphs)

                # Write the content into the file with a separator
                file.write(f"URL: {url}\n{'=' * 80}\n")
                file.write(content + "\n\n")

            except requests.exceptions.RequestException as e:
                print(f"Failed to scrape {url}. Error: {str(e)}")
                file.write(f"URL: {url}\nError: {str(e)}\n\n")

    print(f"Scraping completed. Content saved to '{output_file}'")


In [4]:
planner_agent = AssistantAgent(
    "Planner_Agent",
    system_message="You are a trip planner assistant whose objective is to plan itineraries of the trip to a destination. "
                   "Use tools to fetch weather, search web using bing_search, "
                   "scrape web context for search urls using visit_website tool and "
                   "do RAG on scraped documents to find relevant section of web context to find out accommodation, "
                   "transport, outfits, adventure activities and bookings need. "
                   "Use only the tools provided, and reply TERMINATE when done. "
                   "While executing tools, print outputs and reflect exception if failed to execute a tool. "
                   "If web scraping tool is required, create a temp txt file to store scraped website contents "
                   "and use the same file for rag_on_document as input.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# Setting up the code executor (optional for running generated code snippets)
work_dir = Path("../coding")
work_dir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
# Test Code execution configuration
print(
    code_executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code="print('Hello, World!');"),
        ]
    )
)

# Define the UserProxyAgent
user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)

exit_code=0 output='Hello, World!\n' code_file='/Users/shibsankar/PycharmProjects/LLMAgent/coding/tmp_code_c83c27f1edd76de948b84d7b016bfe4c.py'


In [5]:
# Register tools with the LLM Agent
register_function(
    get_weather_info,
    caller=planner_agent,
    executor=user_proxy,
    name = "get_weather_info",
    description = "This tool fetch weather data from open source api"
)

register_function(
    rag_on_document,
    caller=planner_agent,
    executor=user_proxy,
    name = "rag_on_document",
    description = "This tool fetch relevant information from a document"
)

register_function(
    bing_search,
    caller=planner_agent,
    executor=user_proxy,
    name = "bing_search",
    description = "This tool to search a query in web and get results."
)

register_function(
    visit_website,
    caller=planner_agent,
    executor=user_proxy,
    name = "visit_website",
    description = "This tool is to scrape content of website using a list of urls and store the website content into a text file that can be used for rag_on_document"
)

In [6]:
# The input Question the User want to ask the LLM Agent
question = "Plan a trip to Grand Canyon next month on 16 Nov 2024, I will stay for 5 nights"

In [7]:
ReAct_prompt = """
You are a Trip Planning expert tasked with helping users making a trip itinerary.
You can analyse the query, figure out the travel destination, dates and assess the need of checking weather forecast, search accomodation, recommend outfits and suggest adventure activities like hiking, trekking opportunity and need for advance booking.
Use the following format:

Question: the input question or request
Thought: you should always think about what to do to respond to the question
Action: the action to take (if any)
Action Input: the input to the action (e.g., search query, location for weather, query for rag, url for web scraping)
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question or request
Once get all the answer ask the planner agent to write code and execute to visualise the answer in a table format. 
Begin!
Question: {input}
"""

# Define the ReAct prompt message
def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

with Cache.disk(cache_seed=43) as cache:
    planner_result = user_proxy.initiate_chat(
        planner_agent,
        message=react_prompt_message,
        question=question,
        cache=cache,
        summary_method="reflection_with_llm",
        max_turns=10
    )

user_proxy (to Planner_Agent):


You are a Trip Planning expert tasked with helping users making a trip itinerary.
You can analyse the query, figure out the travel destination, dates and assess the need of checking weather forecast, search accomodation, recommend outfits and suggest adventure activities like hiking, trekking opportunity and need for advance booking.
Use the following format:

Question: the input question or request
Thought: you should always think about what to do to respond to the question
Action: the action to take (if any)
Action Input: the input to the action (e.g., search query, location for weather, query for rag, url for web scraping)
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question or request
Once get all the answer ask the planner agent to write code and execute to visualise the answer in a table format. 
Begin!
Question: Plan a 

2024-10-19 14:03:09,033 INFO:__main__:************  Get weather API is executed for Grand Canyon, 2024-11-16 ************


user_proxy (to Planner_Agent):

user_proxy (to Planner_Agent):

***** Response from calling tool (call_oTWeDD0VF6vu68luAd5CIFvL) *****
{'latitude': 36.10486, 'longitude': -112.11404, 'generationtime_ms': 0.5359649658203125, 'utc_offset_seconds': -25200, 'timezone': 'America/Phoenix', 'timezone_abbreviation': 'MST', 'elevation': 1009.0, 'daily_units': {'time': 'iso8601', 'temperature_2m_max': '°C', 'precipitation_sum': 'mm'}, 'daily': {'time': ['2024-10-19', '2024-10-20', '2024-10-21', '2024-10-22', '2024-10-23', '2024-10-24', '2024-10-25'], 'temperature_2m_max': [19.6, 17.1, 23.0, 25.8, 27.4, 29.0, 28.5], 'precipitation_sum': [7.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}}
**********************************************************************

--------------------------------------------------------------------------------
Planner_Agent (to user_proxy):

Observation: The weather forecast for the Grand Canyon around mid-October shows temperatures ranging from 17.1°C to 29.0°C with some precipitat

/Users/shibsankar/opt/anaconda3/envs/env_llm_agent/lib/python3.9/site-packages/autogen/agentchat/conversable_agent.py:2492: UserWarning: Function 'visit_website' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)



>>>>>>>> EXECUTING FUNCTION informational_web_search...
bing_search (to user_proxy):

Address: bing: Accommodation options near Grand Canyon
Title: Accommodation options near Grand Canyon - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'Accommodation options near Grand Canyon' found 10 results:

## Web Results
1. [The Best Places to Stay Near Grand Canyon National Park](https://www.cntraveler.com/gallery/hotels-near-the-grand-canyon)
The warm desert breeze, dozens of miles of view-lined highways, and mind-blowing rust-red cliffs are just a handful of reasons why the Grand Canyon is a bucket list destination for many Americans ...

2. [Where to Stay In & Near Grand Canyon (Complete Lodging Guide)](https://fullsuitcase.com/grand-canyon-where-to-stay/)
Tusayan Hotels near Grand Canyon. Tusayan, just a 6 miles/ 10 minutes drive from the South Rim, is the best place to stay near Grand Canyon.Its location is still very good for visiting the Grand Canyon, but staying here 


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION bing_search...
user_proxy (to bing_search):

Recommended outfits for Grand Canyon in November

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
bing_search (to user_proxy):

Address: bing: recommended outfits for Grand Canyon in November
Title: recommended outfits for Grand Canyon in November - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'recommended outfits for Grand Canyon in November' found 10 results:

## Web Results
1. [What to wear in Grand Canyon (Packing List + Outfits)](https://wearwhenwhatwhy.com/what-to-wear-in-grand-canyon-packing-list-outfits/)
Find out what to wear and what to pack for Grand Canyon National Park in any season. Get tips on layers, hiking shoes, sun protection, water and more.

2. [Grand Canyon November: 6 expe


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION bing_search...
user_proxy (to bing_search):

Adventure activities in Grand Canyon

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
bing_search (to user_proxy):

Address: bing: Adventure activities in Grand Canyon
Title: Adventure activities in Grand Canyon - Search
Viewport position: Showing page 1 of 1.
A Bing search for 'Adventure activities in Grand Canyon' found 18 results:

## Web Results
1. [20 Epic Things to Do in the Grand Canyon | U.S. News Travel](https://travel.usnews.com/Grand_Canyon_AZ/Things_To_Do/)
2 hours to Half Day. TIME TO SPEND. Read More. Just as the North Kaibab Trail begins at the North Rim, the South Kaibab Trail begins at the South Rim and ventures into the Grand Canyon down to the ...

2. [Free](https://travel.usnews.com/Grand_Canyon



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION visit_page...

>>>>>>>> EXECUTING FUNCTION visit_page...

>>>>>>>> EXECUTING FUNCTION visit_page...
bing_search (to user_proxy):

Address: https://theroguetraveler.com/tours-of-grand-canyon/
Title: 7 Adventure Activities In The Grand Canyon That Are Worth Trying | The Rogue Traveler
Viewport position: Showing page 1 of 5.
7 Adventure Activities In The Grand Canyon That Are Worth Trying \| The Rogue Traveler

[![The Rogue Traveler](/wp-content/uploads/2023/07/2.png)](https://theroguetraveler.com/)

* [Home](https://theroguetraveler.com/)
* [Tour](https://theroguetraveler.com/tour/)
* [Categories](https://theroguetraveler.com/categories/)
* [Blog](https://theroguetraveler.com/custom-blog/)
* [About](https://theroguetraveler.com/about/)
* [DYI Tour](https://theroguetraveler.com/diy-tour/)

 

Select Page

  
 

 

 

7 Adventure Activities In The Gr


--------------------------------------------------------------------------------
user_proxy (to Planner_Agent):

user_proxy (to Planner_Agent):

***** Response from calling tool (call_7tU2CsPCmOT3LUCVbYypwLVA) *****
1. Condé Nast Traveler lists 14 top accommodations near the Grand Canyon, including hotels, Airbnbs, and glamping sites, catering to various preferences such as nature, spas, and fine dining.
2. Full Suitcase provides a complete lodging guide for staying in and near the Grand Canyon, highlighting options in nearby Tusayan and within the park.
3. The National Park Service website details lodging options within Grand Canyon National Park, including South Rim and North Rim accommodations, and provides reservation information.
**********************************************************************

--------------------------------------------------------------------------------
user_proxy (to Planner_Agent):

***** Response from calling tool (call_nxYJS8nhp9yCt3U8PfeW32aC) ****


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
user_proxy (to Planner_Agent):

exitcode: 0 (execution succeeded)
Code output: | Category              | Details                                                                    |
|:----------------------|:---------------------------------------------------------------------------|
| Weather               | Daytime temperatures: ~20°C (68°F)                                         |
|                       | Nighttime temperatures: ~1°C (34°F)                                        |
|                       | Dress in layers to accommodate the varied temperatures throughout the day. |
| Accommodation Options | 1. Condé Nast Traveler Recommendations:                                    |
|                       | - Hotels, Airbnbs, and glamping sites                                      |
|                       | - Options catering to nature

In [8]:
print(f"Summary of the Trip Planning")
print(planner_result.chat_history)
print(planner_result.summary)

Summary of the Trip Planning
[{'content': '\nYou are a Trip Planning expert tasked with helping users making a trip itinerary.\nYou can analyse the query, figure out the travel destination, dates and assess the need of checking weather forecast, search accomodation, recommend outfits and suggest adventure activities like hiking, trekking opportunity and need for advance booking.\nUse the following format:\n\nQuestion: the input question or request\nThought: you should always think about what to do to respond to the question\nAction: the action to take (if any)\nAction Input: the input to the action (e.g., search query, location for weather, query for rag, url for web scraping)\nObservation: the result of the action\n... (this process can repeat multiple times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question or request\nOnce get all the answer ask the planner agent to write code and execute to visualise the answer in a table format. \

In [9]:
pprint.pp(planner_result.chat_history)

[{'content': '\n'
             'You are a Trip Planning expert tasked with helping users making '
             'a trip itinerary.\n'
             'You can analyse the query, figure out the travel destination, '
             'dates and assess the need of checking weather forecast, search '
             'accomodation, recommend outfits and suggest adventure activities '
             'like hiking, trekking opportunity and need for advance booking.\n'
             'Use the following format:\n'
             '\n'
             'Question: the input question or request\n'
             'Thought: you should always think about what to do to respond to '
             'the question\n'
             'Action: the action to take (if any)\n'
             'Action Input: the input to the action (e.g., search query, '
             'location for weather, query for rag, url for web scraping)\n'
             'Observation: the result of the action\n'
             '... (this process can repeat multiple times)\n'

In [10]:
pprint.pp(planner_result.chat_history[-3])

{'content': 'I now have the final answer visualized in a table format.\n'
            '\n'
            'Final Answer:\n'
            '### Trip Itinerary to the Grand Canyon (16th November 2024 - 21st '
            'November 2024)\n'
            '\n'
            '| Category              | '
            'Details                                                                    '
            '|\n'
            '|:----------------------|:---------------------------------------------------------------------------|\n'
            '| **Weather**               | - Daytime temperatures: ~20°C '
            '(68°F)                                         |\n'
            '|                       | - Nighttime temperatures: ~1°C '
            '(34°F)                                        |\n'
            '|                       | - Dress in layers to accommodate the '
            'varied temperatures throughout the day. |\n'
            '| **Accommodation Options** | 1. Condé Nast Traveler '
  